In [1]:
import numpy as np 
import pandas as pd
import keras 
from keras.models import Sequential
from keras.layers import Conv2D, Dense, MaxPool2D, Dropout, Flatten, BatchNormalization
from keras import optimizers
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from skimage import io
from skimage.transform import resize
from keras.utils import np_utils
from sklearn.metrics import accuracy_score
import glob

Using TensorFlow backend.


## Datashape

In [2]:
path_images = ['/Users/flavio/Downloads/Learning/Eric_images/database/SAXS/*.tif',
               '/Users/flavio/Downloads/Learning/Eric_images/database/WAXS/*.tif', 
               '/Users/flavio/Downloads/Learning/Eric_images/database/GIWAXS/*.tif',
               '/Users/flavio/Downloads/Learning/Eric_images/database/GISAXS/*.tif']

learnining_rate_ = 0.01
decay_ = 0.0001
epochs_ = 10
batch_size_ = 32

In [3]:
cont = 0
labels = []
database = []
for i in range(0,len(path_images)):
    image_names = glob.glob(path_images[cont])
    for name in image_names:
        im = io.imread(name)
        im = im.reshape((im.shape[0],im.shape[1],1))
        database.append(im)
        labels.append(cont)
    cont+=1
database = np.array(database,dtype=np.uint8)
labels = np.array(labels,dtype=np.int)
labels = np_utils.to_categorical(labels)

In [4]:
X_train, X_test, y_train, y_test = train_test_split(database,labels, test_size= 0.2)

In [5]:
model = Sequential()

model.add(Conv2D(filters=64, kernel_size=(3,3), padding='same', activation='relu', input_shape=(X_train.shape[1], X_train.shape[2], X_train.shape[3])))
model.add(BatchNormalization())
model.add(MaxPool2D(strides=2))

model.add(Conv2D(filters=64, kernel_size=(3,3), padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(MaxPool2D(strides=2))

model.add(Conv2D(filters=48, kernel_size=(3,3), padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(MaxPool2D(strides=2))

model.add(Flatten())

model.add(Dense(192, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dense(4, activation='softmax'))

In [6]:
model.build()
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 256, 256, 64)      640       
_________________________________________________________________
batch_normalization_1 (Batch (None, 256, 256, 64)      256       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 128, 128, 64)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 128, 128, 64)      36928     
_________________________________________________________________
batch_normalization_2 (Batch (None, 128, 128, 64)      256       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 64, 64, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 64, 64, 48)       

In [7]:
sgd = optimizers.SGD(lr=learnining_rate_,
                    decay=decay_)
model.compile(sgd, loss='categorical_crossentropy', metrics=['accuracy'])

In [8]:
model.fit(X_train, y_train, batch_size=batch_size_, epochs=epochs_)

Epoch 1/10
320/320 [==============================] - 65s 204ms/step - loss: 4.8232 - accuracy: 0.5656
Epoch 2/10
320/320 [==============================] - 67s 210ms/step - loss: 0.7470 - accuracy: 0.7781
Epoch 3/10
320/320 [==============================] - 54s 168ms/step - loss: 0.3310 - accuracy: 0.8844
Epoch 4/10
320/320 [==============================] - 54s 167ms/step - loss: 0.2419 - accuracy: 0.9094
Epoch 5/10
320/320 [==============================] - 54s 169ms/step - loss: 0.1961 - accuracy: 0.9281
Epoch 6/10
320/320 [==============================] - 55s 172ms/step - loss: 0.2120 - accuracy: 0.9344
Epoch 7/10
320/320 [==============================] - 54s 170ms/step - loss: 0.1139 - accuracy: 0.9688
Epoch 8/10
320/320 [==============================] - 54s 170ms/step - loss: 0.0839 - accuracy: 0.9812
Epoch 9/10
320/320 [==============================] - 55s 171ms/step - loss: 0.1009 - accuracy: 0.9625
Epoch 10/10
320/320 [==============================] - 55s 173ms/step - l

In [10]:
result = model.predict(X_test)

In [11]:
result = result.argmax(axis=1)

In [12]:
print('Accuracy: ',accuracy_score(y_test.argmax(axis=1), result))

Accuracy:  0.95


In [13]:
model.save('lenet.h5')